# Python und MongoDB

Dieses Beispiel setzt einen laufenden MongoDB-Server mit aktivierter Authentifizierung voraus. 

## Erstellen der Verbindung zum Datenbank Server 

Um aus Python auf eine NoSQL-Datenbank (hier MongoDB) zugreifen zu können, muss die entsprechende Bibliothek eingebunden werden. 

In [1]:
import pymongo

Definition der Datenbank-Parameter als Konstanten

Hinweis: In Docker mit "docker inspect Mongo_Server" die IP-Adresse ermitteln.

In [2]:
DATABASE_HOST      =  "192.168.0.30"
DATABASE_PORT      =  "27017"
DATABASE_USER      =  "mongoAdmin"
DATABASE_PASSWORT  =  "pkmlp"
DATABASE_NAME      =  "meineDatenbank"
COLLECTION_NAME    =  "meineSammlung"
AUTH_DATABASE      =  "admin"

Als nächstes muss eine Verbindung zur Datenbank (mit gültigen Credentials) hergestellt werden.  

In [3]:
try:
    dbVerbindung = pymongo.MongoClient('mongodb://'+DATABASE_USER+':'+DATABASE_PASSWORT+'@'+
                                                    DATABASE_HOST+':'+DATABASE_PORT+'/'+AUTH_DATABASE)
    print("\nVerbindung zu MongoDB erstellt\n")
except pymongo.errors.ConnectionFailure as VerbindungsFehler:
    print("\nKeine Verbindung zu MongoDB: ", VerbindungsFehler, " - Programmabbruch\n")


Verbindung zu MongoDB erstellt



## Auswählen der Datenbank und der Sammlung

Verbindung konnte erstellt werden, nun die Datenbank ...

In [4]:
db = dbVerbindung[DATABASE_NAME]

... und die Collection in der Datenbank wählen

In [5]:
sammlung = db[COLLECTION_NAME]

### Kontrolliere in MongoClient, MongoBooster oder im mongo Terminal ob die Collection wirklich leer ist.

Die Collection muss nicht zwingend leer sein. Ist sie es nicht, so muss der Anfangs-Datenbestand berücksichtigt werden. Das heisst, die nachfolgend eingefügten (Create), gelesenen (Read), mutierten (Update) und gelöschten (Delete) Dokumente sind immer auch mit den allenfalls bereits vorhandenen Dokumenten "zu sehen". Dieses Beispiel ist besser zu Interpretieren, wenn die Sammlung vor Beginn leer ist.

Definieren von 4 einzelnen Dokumenten. Beachte: die Dokumente müssen nicht eine einheitliche Struktur (gleiche Attribute) haben. 

In [6]:
dokument_1 = {"Name":"Kessler", "Vorname":"Peter", "Mail":"peter@trueworld.ch"}
dokument_2 = {"Name":"Kessler", "Vorname":"Paul",  "Mail":"paul@trueworld.ch"}
dokument_3 = {"Name":"Meier",   "Vorname":"peter", "Mail":"meier@trueworld.ch"}
dokument_4 = {"Name":"Enz",     "Vorname":"Hugo",  "Mobile":"044 333 22 11"}

## Einfügen von Dokumenten in die Sammlung (Collection) der Datenbank

Einfügen der einzeln definierten Dokumente in die Datenbank/Collection

In [7]:
insertedKey = sammlung.insert_one(dokument_1)
print("\nPrimekey des eingefügten Dokumentes: ", insertedKey.inserted_id)

insertedKey = sammlung.insert_one(dokument_2)
print("Primekey des eingefügten Dokumentes: ", insertedKey.inserted_id)

insertedKey = sammlung.insert_one(dokument_3)
print("Primekey des eingefügten Dokumentes: ", insertedKey.inserted_id)

insertedKey = sammlung.insert_one(dokument_4)
print("Primekey des eingefügten Dokumentes: ", insertedKey.inserted_id)


Primekey des eingefügten Dokumentes:  5c4af39608583800149aecb6
Primekey des eingefügten Dokumentes:  5c4af39608583800149aecb7
Primekey des eingefügten Dokumentes:  5c4af39608583800149aecb8
Primekey des eingefügten Dokumentes:  5c4af39608583800149aecb9


Definieren einer Liste mit mehreren Dokumenten

In [8]:
sammelDokument = [
    {"Name":"Müller",     "Vorname":"Ralf",  "Mail":"ralf@pkmlp.ch"},
    {"Name":"Hanselmann", "Vorname":"Karl",  "Mail":"khanselmann@pkmlp.ch"},
    {"Name":"Bitbeisser", "Vorname":"Bodo",  "Mail":"bodo@bitbeisser.ch"},
    {"Name":"Consani",    "Vorname":"Fabio", "Mobile":"044 999 88 77"}
]

Einfügen der Liste mit den Dokumenten in die Datenbank/Collection

In [9]:
insertedKeys = sammlung.insert_many(sammelDokument)
print("\nArray der Primekeys der eingefügten Dokumente: ")
print(insertedKeys.inserted_ids)

print("\nPrimekeys der eingefügten Dokumente einzeln: ")
for i in range(0, len(insertedKeys.inserted_ids)):
    print(insertedKeys.inserted_ids[i])


Array der Primekeys der eingefügten Dokumente: 
[ObjectId('5c4af39b08583800149aecba'), ObjectId('5c4af39b08583800149aecbb'), ObjectId('5c4af39b08583800149aecbc'), ObjectId('5c4af39b08583800149aecbd')]

Primekeys der eingefügten Dokumente einzeln: 
5c4af39b08583800149aecba
5c4af39b08583800149aecbb
5c4af39b08583800149aecbc
5c4af39b08583800149aecbd


### Kontrolliere in MongoClient, MongoBooster oder im mongo Terminal die Collection.

War die Sammlung zu Beginn leer, so dürfen jetzt nur die 8 obigen Dokumente darin sein. War die Sammlung zu Beginn nicht leer, so sind jetzt zusätzlich zu den bereits vorhandenen Dokumenten die obigen 8 in der Sammlung.

## Suchen und Zählen von Dokumenten in der Sammlung

In [10]:
anzDokumente = sammlung.find().count()
print("Die Sammlung enthält",anzDokumente, "Dokumente\n")

Die Sammlung enthält 8 Dokumente



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


Zählen der Dokumente mit einer Mailadresse

In [11]:
anzDokumente = sammlung.find({"Mail":{"$exists":True}}).count()
print("Die Sammlung enthält",anzDokumente, "Dokumente mit Mailadresse\n")

Die Sammlung enthält 6 Dokumente mit Mailadresse



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


Zählen der Dokumente die keine Mailadfresse enthalten

In [12]:
anzDokumente = sammlung.find({"Mail":{"$exists":False}}).count()
print("Die Sammlung enthält",anzDokumente, "Dokumente ohne Mailadresse\n")

Die Sammlung enthält 2 Dokumente ohne Mailadresse



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


Zählen der Dokumente die im Attribut (Field) Vorname "Peter" enthalten

In [13]:
anzDokumente = sammlung.find({"Vorname":"Peter"}).count()
print("Davon entsprechen",anzDokumente,"dem Suchkriterium: Vorname = Peter\n")

Davon entsprechen 1 dem Suchkriterium: Vorname = Peter



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


Suchen und zählen von Dokumenten in der Datenbank/Collection

In [14]:
suchKriterium = {"Name":"Kessler"}
anzDokumente = sammlung.find(suchKriterium).count()
print("Davon entsprechen",anzDokumente,"dem Suchkriterium:",suchKriterium,"\n")

Davon entsprechen 2 dem Suchkriterium: {'Name': 'Kessler'} 



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


In [15]:
suchKey = "Name"
suchWert = "Kessler"
anzDokumente = sammlung.find({suchKey:suchWert}).count()
print("Davon entsprechen",anzDokumente,"dem Suchkriterium:",suchKey,"=",suchWert,"\n")

Davon entsprechen 2 dem Suchkriterium: Name = Kessler 



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
suchKey = "Vorname"
suchWert = "Hugo"
suchKriterium = {suchKey:suchWert}
anzDokumente = sammlung.find(suchKriterium).count()
print("Davon entsprechen",anzDokumente,"dem Suchkriterium:",suchKey,"=",suchWert,"\n")

Davon entsprechen 1 dem Suchkriterium: Vorname = Hugo 



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


In [17]:
suchKey = "Vorname"
suchWert = "Paul"
print("Davon entsprechen",sammlung.find({suchKey:suchWert}).count(),
      "dem Suchkriterium:",suchKey,"=",suchWert,"\n")

Davon entsprechen 1 dem Suchkriterium: Vorname = Paul 



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


## Lesen und Ausgeben von Dokumenten in der Sammlung 

Lesen aller Dokumente aus der Datenbank
Hinweis: Die Reihenfolge der einzelnen Attribute in einer Zeile sind nicht definiert.
Darum müssen Attribute gezielt mit dem Key aus dem Cursor gelesen werden.

In [18]:
print("Lesen aller Dokumente nach Einfuegen")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)

Lesen aller Dokumente nach Einfuegen
{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Vorname': 'Peter', 'Mail': 'peter@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Vorname': 'Paul', 'Mail': 'paul@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb8'), 'Name': 'Meier', 'Vorname': 'peter', 'Mail': 'meier@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb9'), 'Name': 'Enz', 'Vorname': 'Hugo', 'Mobile': '044 333 22 11'}
{'_id': ObjectId('5c4af39b08583800149aecba'), 'Name': 'Müller', 'Vorname': 'Ralf', 'Mail': 'ralf@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbb'), 'Name': 'Hanselmann', 'Vorname': 'Karl', 'Mail': 'khanselmann@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbc'), 'Name': 'Bitbeisser', 'Vorname': 'Bodo', 'Mail': 'bodo@bitbeisser.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbd'), 'Name': 'Consani', 'Vorname': 'Fabio', 'Mobile': '044 999 88 77'}


In [19]:
print("\nLesen aller Dokumente aber nur Attributswerte ausgeben")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument.get("Name"), dokument.get("Vorname"), dokument.get("Mail", "keine Mailadresse"))


Lesen aller Dokumente aber nur Attributswerte ausgeben
Kessler Peter peter@trueworld.ch
Kessler Paul paul@trueworld.ch
Meier peter meier@trueworld.ch
Enz Hugo keine Mailadresse
Müller Ralf ralf@pkmlp.ch
Hanselmann Karl khanselmann@pkmlp.ch
Bitbeisser Bodo bodo@bitbeisser.ch
Consani Fabio keine Mailadresse


In [20]:
print("\nLesen aller Dokumente absteigend sortiert nach Vorname")
dokumentCursor = sammlung.find().sort("Vorname",-1)
for dokument in dokumentCursor:
    print(dokument.get("Name"), dokument.get("Vorname"), dokument.get("Mail", "keine Mailadresse"))


Lesen aller Dokumente absteigend sortiert nach Vorname
Meier peter meier@trueworld.ch
Müller Ralf ralf@pkmlp.ch
Kessler Peter peter@trueworld.ch
Kessler Paul paul@trueworld.ch
Hanselmann Karl khanselmann@pkmlp.ch
Enz Hugo keine Mailadresse
Consani Fabio keine Mailadresse
Bitbeisser Bodo bodo@bitbeisser.ch


## Lesen/Suchen und Ausgeben von Dokumenten 

Lesen von Dokumenten mit Suchkriterium 'Vorname = Peter'

In [21]:
dokumentCursor = sammlung.find({"Vorname":"Peter"})
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Vorname': 'Peter', 'Mail': 'peter@trueworld.ch'}


In [22]:
suchKriterium = {"Name":"Kessler"}
dokumentCursor = sammlung.find(suchKriterium)
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Vorname': 'Peter', 'Mail': 'peter@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Vorname': 'Paul', 'Mail': 'paul@trueworld.ch'}


Lesen von Dokumenten mit Suchkriterium 'Vorname ungleich Peter'

In [23]:
dokumentCursor = sammlung.find({"Vorname":{"$ne":"Peter"}})
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Vorname': 'Paul', 'Mail': 'paul@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb8'), 'Name': 'Meier', 'Vorname': 'peter', 'Mail': 'meier@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb9'), 'Name': 'Enz', 'Vorname': 'Hugo', 'Mobile': '044 333 22 11'}
{'_id': ObjectId('5c4af39b08583800149aecba'), 'Name': 'Müller', 'Vorname': 'Ralf', 'Mail': 'ralf@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbb'), 'Name': 'Hanselmann', 'Vorname': 'Karl', 'Mail': 'khanselmann@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbc'), 'Name': 'Bitbeisser', 'Vorname': 'Bodo', 'Mail': 'bodo@bitbeisser.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbd'), 'Name': 'Consani', 'Vorname': 'Fabio', 'Mobile': '044 999 88 77'}


In [24]:
suchKriterium = {"Vorname":{"$ne":"Peter"}}
dokumentCursor = sammlung.find(suchKriterium)
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Vorname': 'Paul', 'Mail': 'paul@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb8'), 'Name': 'Meier', 'Vorname': 'peter', 'Mail': 'meier@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb9'), 'Name': 'Enz', 'Vorname': 'Hugo', 'Mobile': '044 333 22 11'}
{'_id': ObjectId('5c4af39b08583800149aecba'), 'Name': 'Müller', 'Vorname': 'Ralf', 'Mail': 'ralf@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbb'), 'Name': 'Hanselmann', 'Vorname': 'Karl', 'Mail': 'khanselmann@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbc'), 'Name': 'Bitbeisser', 'Vorname': 'Bodo', 'Mail': 'bodo@bitbeisser.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbd'), 'Name': 'Consani', 'Vorname': 'Fabio', 'Mobile': '044 999 88 77'}


Lesen von Dokumenten mit Suchkriterien 'Vorname = Peter und Name = Kessler'

In [25]:
dokumentCursor = sammlung.find({"Vorname":"Peter","Name":"Kessler"})
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Vorname': 'Peter', 'Mail': 'peter@trueworld.ch'}


In [26]:
suchKriterium = {"Vorname":"Peter","Name":"Kessler"}
dokumentCursor = sammlung.find(suchKriterium)
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Vorname': 'Peter', 'Mail': 'peter@trueworld.ch'}


Lesen von Dokumenten mit Suchkriterium 'Name beginnt mit K'

In [27]:
dokumentCursor = sammlung.find({"Name":{"$gt":"K","$lt":"L"}})
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Vorname': 'Peter', 'Mail': 'peter@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Vorname': 'Paul', 'Mail': 'paul@trueworld.ch'}


In [28]:
suchKriterium = {"Name":{"$gt":"K","$lt":"L"}}
dokumentCursor = sammlung.find(suchKriterium)
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Vorname': 'Peter', 'Mail': 'peter@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Vorname': 'Paul', 'Mail': 'paul@trueworld.ch'}


## Lesen ausgewählter Attribute (Fields) der Dokumente aus der Sammlung 

Lesen ausgewählter Attribute der Dokumente aus der Datenbank:
Hinweis: Der Primekey _id wird immer gelesen und zurückgegeben,
wenn dieser nicht ausdrücklich ausgeschlossen wird.

In [29]:
dokumentCursor = sammlung.find({},{"Name":1,"Mail":1})
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Mail': 'peter@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Mail': 'paul@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb8'), 'Name': 'Meier', 'Mail': 'meier@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb9'), 'Name': 'Enz'}
{'_id': ObjectId('5c4af39b08583800149aecba'), 'Name': 'Müller', 'Mail': 'ralf@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbb'), 'Name': 'Hanselmann', 'Mail': 'khanselmann@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbc'), 'Name': 'Bitbeisser', 'Mail': 'bodo@bitbeisser.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbd'), 'Name': 'Consani'}


In [30]:
dokumentCursor = sammlung.find({},{"Name":True,"Mail":True})
for dokument in dokumentCursor:
    print(dokument)

{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Mail': 'peter@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Mail': 'paul@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb8'), 'Name': 'Meier', 'Mail': 'meier@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb9'), 'Name': 'Enz'}
{'_id': ObjectId('5c4af39b08583800149aecba'), 'Name': 'Müller', 'Mail': 'ralf@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbb'), 'Name': 'Hanselmann', 'Mail': 'khanselmann@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbc'), 'Name': 'Bitbeisser', 'Mail': 'bodo@bitbeisser.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbd'), 'Name': 'Consani'}


In [31]:
dokumentCursor = sammlung.find({},{"Name":1,"Mail":1,"_id":0})
for dokument in dokumentCursor:
    print(dokument)

{'Name': 'Kessler', 'Mail': 'peter@trueworld.ch'}
{'Name': 'Kessler', 'Mail': 'paul@trueworld.ch'}
{'Name': 'Meier', 'Mail': 'meier@trueworld.ch'}
{'Name': 'Enz'}
{'Name': 'Müller', 'Mail': 'ralf@pkmlp.ch'}
{'Name': 'Hanselmann', 'Mail': 'khanselmann@pkmlp.ch'}
{'Name': 'Bitbeisser', 'Mail': 'bodo@bitbeisser.ch'}
{'Name': 'Consani'}


In [32]:
dokumentCursor = sammlung.find({},{"Name":True,"Mail":True,"_id":False})
for dokument in dokumentCursor:
    print(dokument)

{'Name': 'Kessler', 'Mail': 'peter@trueworld.ch'}
{'Name': 'Kessler', 'Mail': 'paul@trueworld.ch'}
{'Name': 'Meier', 'Mail': 'meier@trueworld.ch'}
{'Name': 'Enz'}
{'Name': 'Müller', 'Mail': 'ralf@pkmlp.ch'}
{'Name': 'Hanselmann', 'Mail': 'khanselmann@pkmlp.ch'}
{'Name': 'Bitbeisser', 'Mail': 'bodo@bitbeisser.ch'}
{'Name': 'Consani'}


## Mutieren (ändern) von Dokumenten in der Sammlung

Updaten eines ganz bestimmten Dokumentes in der Sammlung und zur Kontrolle vor- und nachher alles ausgeben.

In [33]:
print("\nLesen aller Dokumente vor Update")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)
    
print("\nUpdate des Dokumentes: peter --> Peter")
sammlung.update_one({"Vorname":"peter"},{"$set":{"Vorname":"Peter"}})

print("\nLesen aller Dokumente nach Update")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)


Lesen aller Dokumente vor Update
{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Vorname': 'Peter', 'Mail': 'peter@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Vorname': 'Paul', 'Mail': 'paul@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb8'), 'Name': 'Meier', 'Vorname': 'peter', 'Mail': 'meier@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb9'), 'Name': 'Enz', 'Vorname': 'Hugo', 'Mobile': '044 333 22 11'}
{'_id': ObjectId('5c4af39b08583800149aecba'), 'Name': 'Müller', 'Vorname': 'Ralf', 'Mail': 'ralf@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbb'), 'Name': 'Hanselmann', 'Vorname': 'Karl', 'Mail': 'khanselmann@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbc'), 'Name': 'Bitbeisser', 'Vorname': 'Bodo', 'Mail': 'bodo@bitbeisser.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbd'), 'Name': 'Consani', 'Vorname': 'Fabio', 'Mobile': '044 999 88 77'}

Update des Dokumentes: peter --> Peter

Lesen aller Dokumente

### Kontrolliere in MongoClient, MongoBooster oder im mongo Terminal die Collection.

## Löschen von Dokumenten in der Sammlung 

Löschen eines Dokumentes aus der Sammlung

In [34]:
print("\nLesen aller Dokumente vor Delete")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)
    
print("\nLöschen des Dokumentes: Peter Kessler")
anzDelete = sammlung.delete_one({"Vorname":"Peter", "Name":"Kessler"})
print("Anzahl gelöschter Dokumente: ", anzDelete.deleted_count)

print("\nLesen aller Dokumente nach Delete")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)


Lesen aller Dokumente vor Delete
{'_id': ObjectId('5c4af39608583800149aecb6'), 'Name': 'Kessler', 'Vorname': 'Peter', 'Mail': 'peter@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Vorname': 'Paul', 'Mail': 'paul@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb8'), 'Name': 'Meier', 'Vorname': 'Peter', 'Mail': 'meier@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb9'), 'Name': 'Enz', 'Vorname': 'Hugo', 'Mobile': '044 333 22 11'}
{'_id': ObjectId('5c4af39b08583800149aecba'), 'Name': 'Müller', 'Vorname': 'Ralf', 'Mail': 'ralf@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbb'), 'Name': 'Hanselmann', 'Vorname': 'Karl', 'Mail': 'khanselmann@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbc'), 'Name': 'Bitbeisser', 'Vorname': 'Bodo', 'Mail': 'bodo@bitbeisser.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbd'), 'Name': 'Consani', 'Vorname': 'Fabio', 'Mobile': '044 999 88 77'}

Löschen des Dokumentes: Peter Kessler
Anzahl gelöschter Dokum

Löschen aller Dokumente aus der Datenbank

In [35]:
print("\nLesen aller Dokumente vor Delete all")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)
    
print("\nLöschen aller Dokumente")
anzDelete = sammlung.delete_many({})
print("Anzahl gelöschter Dokumente: ", anzDelete.deleted_count)

print("\nLesen aller Dokumente nach Delete all")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)


Lesen aller Dokumente vor Delete all
{'_id': ObjectId('5c4af39608583800149aecb7'), 'Name': 'Kessler', 'Vorname': 'Paul', 'Mail': 'paul@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb8'), 'Name': 'Meier', 'Vorname': 'Peter', 'Mail': 'meier@trueworld.ch'}
{'_id': ObjectId('5c4af39608583800149aecb9'), 'Name': 'Enz', 'Vorname': 'Hugo', 'Mobile': '044 333 22 11'}
{'_id': ObjectId('5c4af39b08583800149aecba'), 'Name': 'Müller', 'Vorname': 'Ralf', 'Mail': 'ralf@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbb'), 'Name': 'Hanselmann', 'Vorname': 'Karl', 'Mail': 'khanselmann@pkmlp.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbc'), 'Name': 'Bitbeisser', 'Vorname': 'Bodo', 'Mail': 'bodo@bitbeisser.ch'}
{'_id': ObjectId('5c4af39b08583800149aecbd'), 'Name': 'Consani', 'Vorname': 'Fabio', 'Mobile': '044 999 88 77'}

Löschen aller Dokumente
Anzahl gelöschter Dokumente:  7

Lesen aller Dokumente nach Delete all


### Kontrolliere in MongoClient, MongoBooster oder im mongo Terminal ob die Collection wirklich leer ist.

Verbindung zur Datenbank beenden

In [36]:
dbVerbindung.close()

That's all Folks